**Import Necessary Libraries**

In [ ]:
#cv2
import cv2
#unused but has potential to be used
#import matplotlib.pyplot as plt
#import numpy as np

#cvzone
from cvzone.HandTrackingModule import HandDetector

General Variables

In [ ]:
#Maximum Hand number that we want to track.
maxHandCount= 2

#Colors for ease of use. Cv2 is BGR formatted
black=(0,0,0)
blue = (255,0,0)
green = (0,255,0)
red= (0,0,255)
white=(255,255,255)

#Offset for cropping the image.
cropOffSet = 40

#Regulary used fonts to spot some features.
font = cv2.FONT_HERSHEY_SIMPLEX


Library Variables

In [ ]:
# Initialize Capture
cap = cv2.VideoCapture(1)

#Define hand detector
detector = HandDetector(maxHands=maxHandCount)

Defined Functions

In [ ]:
def drawHandNames(img,left,right):
    centerLeft= left['center']
    centerRight= right['center']
    cv2.putText(img, left["type"], centerLeft, font, 1, red, 2)
    cv2.putText(img, right["type"], centerRight, font, 1, blue, 2)

def minMaxControl(value,max,min=0):
    if value <= min:
        return min
    elif value >= max:
        return max
    else:
        return value

**Determine Hand Type Functionality**
Algoritma kameraya giren elleri farklı algılayabiliyor. Koordinatların doğru tespiti için net bir şekilde sağ ve sol eli bilmem gerekiyor.

In [ ]:
def determineHandType(hand1,hand2):
    """
    Returns left and right hand clearly.

    Args:
        hand1 (dict): hand1 that algorithm caught.
        hand2 (dict): hand2 that algorithm caught.

    Returns:
        dict,dict: left,right hands.
    """
    if hand1["type"]=="Left":
        left= hand1
        right= hand2
    else:
        left= hand2
        right= hand1
    return left,right

**Check State Functionality**
    Sol ve Sağ elin birbirine göre koordinatlarının farklı durumlarını 
hesaplamamı sağlıyor

In [ ]:
def checkState(left,right):
    """
    Returns right hands relative location to left hand.

    Args:
        left (dict): x,y of Left Hand.
        right (dict): x,y of Right Hand.

    Returns:
        string: Relative Location.
    """
    if (right["x"]>left["x"]):
        #Right 3 possibility
        return "Right"+ checkStateY(left,right)
    elif (right["x"]<left["x"]):
        #Left 3 possibility
        return "Left"+ checkStateY(left,right)
    else:
        #Mid 3 possibility
        return "Mid"+ checkStateY(left,right)

def checkStateY(left,right):
    """
    Returns right hands relative Y(vertical) location to left hand.
    
    Args:
        left (dict): x,y,w,h of Left Hand's bounding box.
        right (dict): x,y,w,h of Right Hand's bounding box.
        Only X and Y values are used.
    Returns:
        string: Relative Y Location.
    """
    if (right["y"]<left["y"]):
        return "Up"
    elif (right["y"]>left["y"]):
        return "Down"
    else:
        return "Mid"

def setTopLeft_BottomRight(state,left,right):
    """
    Return TopLeft and BottomRight values according to the position state.
    
    Args:
        left (dict): x,y,w,h of Left Hand's bounding box.
        right (dict): x,y,w,h of Right Hand's bounding box.

    Returns:
        string: Relative Y Location.
    """
    if (state == "LeftUp" or state == "LeftMid" or state == "MidUp" or state == "MidMid"):
        TopLeft = (right["x"],right["y"])
        BottomRight= (left["x"]+left["w"], left["y"]+left["h"])
    elif (state == "RightDown" or state == "RightMid" or state =="MidDown"):
        TopLeft = (left["x"],left["y"])
        BottomRight= (right["x"]+right["w"], right["y"]+right["h"])
    elif (state == "LeftDown"):
        TopLeft = (right["x"],left["y"])
        BottomRight= (left["x"]+left["w"], right["y"]+right["h"])
    elif (state == "RightUp"):
        TopLeft = (left["x"],right["y"])
        BottomRight= (right["x"]+right["w"], left["y"]+left["h"])

    TopLeft = (TopLeft[0]-cropOffSet,TopLeft[1]-cropOffSet)
    BottomRight = (BottomRight[0]+cropOffSet,BottomRight[1]+cropOffSet)
    return TopLeft,BottomRight

Core Loop

In [ ]:
while True:
    # Kameradan veri oku
    success, img = cap.read()
    # Break if capture fails
    if not success:
        print("Couldn't read any data from capture")
        break


    maxVertical = img.shape[0]
    maxHorizontal = img.shape[1]

    # Görüntüde elleri tespit et
    hands, img = detector.findHands(img)

    if len(hands)==2:
        cv2.putText(img, f"{maxVertical,maxHorizontal}", (10, 50), font, 1, green, 2)

        #Determine the Hand Types Clearly
        left,right = determineHandType(hands[1],hands[0])


        x, y, w, h = left["bbox"]
        leftCoordinates= {"x":x,"y":y,"w":w,"h":h}
        x, y, w, h = right["bbox"]
        rightCoordinates= {"x":x,"y":y,"w":w,"h":h}
        # Adjust coordinates to ensure they are non-negative


        TopLeftCoordinates = {"x":minMaxControl(value=rightCoordinates["x"],max=maxHorizontal),
                              "y":minMaxControl(value=rightCoordinates["y"],max=maxVertical)}
        
        BottomRightCoordinates = {"x":minMaxControl(value=leftCoordinates["x"]+leftCoordinates["w"],max=maxHorizontal),
                                  "y":minMaxControl(value=leftCoordinates["y"]+leftCoordinates["h"], max=maxVertical)}
        

        state=checkState(leftCoordinates,rightCoordinates)
        cv2.putText(img,f"{state}",(rightCoordinates["x"],rightCoordinates["y"]),font,1,green,2)

        TopLeftCoordinates, BottomRightCoordinates = setTopLeft_BottomRight(state,leftCoordinates,rightCoordinates)

        cv2.circle(img, TopLeftCoordinates, radius=5, color=white, thickness=-1)
        cv2.circle(img, BottomRightCoordinates, radius=5, color=black, thickness=-1)

        croppedIMG = img[TopLeftCoordinates[1]:BottomRightCoordinates[1],
                         TopLeftCoordinates[0]:BottomRightCoordinates[0]]
        

        ####
        ####
        ####
        ####
        #Şu an 2 boyutlu dikdörtgenlerin birbirine göre durumlarını hesaplayan 
        #algoritma sayesinde image'ı başarılı bir şekilde kesebiliyoruz.
        #
        #
        #Şimdi ise kestiğimiz bu imageların boyutunu her durumda eşitlememiz gerekiyor.
        #Modele verilmeden önce modele vereceğimiz bu verilerin boyutunu aynı ayarlamamız gerek
        #
        #
        #
        ####
        ####
        ####
        ####
    else:
        croppedIMG=img




    # Görüntüyü göster
    cv2.imshow("Camera", img)
    cv2.imshow("Cropped",croppedIMG)

    # Wait for key press for a 1 ms
    key = cv2.waitKey(1)
    
    # If '0' key pressed then break
    if key == ord('0'):
        break

# Release all cameras and destroy openCV windows.
cap.release()
cap.release()